In [1]:
from nltk.tokenize import sent_tokenize
from gensim.models import Word2Vec
import nltk
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
import glob
import os
from os import path
import string
import numpy
import re
from __future__ import absolute_import, division, print_function
import pandas as pd
import multiprocessing
import gensim
import random
from tqdm import tqdm
import pandas as pd
from gensim.models import KeyedVectors
from numpy import array
import matplotlib.pyplot as plt
import numpy as np

C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
os.chdir('C://Users//Ruben//Documents//GitHub//TheForeign//SCC//output-data')
dfo = pd.DataFrame()

for vocab in ['vreemde', 'buitenlandsche', 'binnenlandsche', 'vaderlandsche', 'europeesche', 'nationale', 'internationale']:
    fn = "bigrams_" + vocab + ".csv"
    df = pd.read_csv(fn)
    df = df[['ngram', '1895']]
    df = df[df['1895'] > 0]
    
    df = df.reset_index(drop=True)
    for c,i in enumerate(df.ngram):
    
        target = i
        source = vocab
        weight = df['1895'][c]
        
        t = pd.DataFrame([source, target, weight]).T
        dfo = dfo.append(t)

In [3]:
dfo.columns = ['source', 'target', 'weight']

In [4]:
os.chdir('C://Users//Ruben//Desktop')

dfo = dfo[~dfo.source.str.contains(r'\d')]
dfo = dfo[~dfo.target.str.contains(r'\d')]

dgr = dict(dfo.target.value_counts())
dgrl = [k for k,v in dgr.items() if v > 1]

dfo = dfo[dfo.target.isin(dgrl)]

In [16]:
total_list_targets = list(dfo.target)


op = pd.DataFrame()

for source in list(set(dfo.source)):
    tmpdf = dfo[dfo.source == source]
    list_source = list(tmpdf.target)
    
    lmcs = list(set(dfo.source)) 
    lmcs.remove(source)
    
    for other_source in lmcs:
        tmpdf_other = dfo[dfo.source == other_source]
        list_other_source = list(tmpdf_other.target)
        
        overlap = len([w for w in list_source if w in list_other_source]) / (len(list_source) + len(list_other_source)) * 100
        
        t = pd.DataFrame([source, other_source, overlap]).T
        op = op.append(t)

In [19]:
op.columns = ['source', 'target', 'weight']
op = op.drop_duplicates(subset=['weight'], keep=False)

In [20]:
os.chdir('C://Users//Ruben//Desktop')

op.to_csv('test_big.csv', index=False)

In [21]:
op

,source,target,weight
0,vreemde,nationale,10.7266
0,vreemde,europeesche,18.1467
0,nationale,vreemde,12.8028
0,nationale,internationale,23.6287
0,nationale,buitenlandsche,9.60265
0,nationale,binnenlandsche,8.4507
0,nationale,vaderlandsche,13.1148
0,nationale,europeesche,8.92857
0,internationale,nationale,19.4093
0,internationale,europeesche,8.69565
